## Digitale Techniken: Weatherstation - correcting time stamps

For the purpose of correcting timestamps in data files measured with arduino weatherstation where for each new measurement the clock is accidentally initialized with the time of compilation.

joker, 2022-01

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" style="height:50px" align="left"/> <br><br>

https://creativecommons.org/licenses/by-nc-sa/4.0/

The example data from WiSe2021/22 for which this script was written comes with this header:<br>
```datetime,lightIntensity,temperature,relHumidity,ppmCO2```

The WiSe2022/23 header looks like this:<br>
```Datetime,Brightness,Relative humidity,Temperature,C02 mixing ratio,Air pressure,Altitude```

#### Below, check (and change):
- the path to your incorrect data files
- the index to pick one of the files
- the true time you want to correct to

In [1]:
# import module
from datetime import datetime, timedelta
from glob import glob
import os

In [2]:
# read data files with incorrect timestamps from a folder "data_incorrect_time"
path = os.path.join('data_incorrect_time', '*.txt')
files = glob(path)
files

['data_incorrect_time/01051302.txt', 'data_incorrect_time/01051302_1.txt']

In [3]:
# choose a file from all incorrect files: CHANGE INDEX
# or use a loop for all files

file = files[0]
filename = str.split(os.path.split(file)[-1],".")[0]
# print(filename)

# provide the correct time of the first measurement to which you want to shift the times: JUST ENTER "2022-..."
# as string in format YYYY-mm-dd HH:MM:SS
true_time = datetime.strptime("2022-01-06 11:12:13", "%Y-%m-%d %H:%M:%S")
print('True start time: ' + str(true_time))
outfile = datetime.strftime(true_time,"%m%d%H%M") + '_corrected_time.txt'

# define number of header lines to skip, separator of data columns
skip = 1
sep = ","
    
with open(file, "r") as f:
    lines = f.readlines()
    # print(lines[0] + lines[1])  # just test printing the first two lines
    
# initialize a new list to hold the corrected lines
lines_new = list()

# loop over all lines while using enumerate() to generate a tuple of line and its number
for line_nr, line in enumerate(lines):
    if line_nr < skip:
        lines_new.append(line)
        continue

    cols = line.split(sep)
    # first column holds time data as strings
    # strptime parses this string to a datetime object in the time format provided 
    # (which has to match the format in the string)
    time = datetime.strptime(cols[0], "%Y-%m-%d %H:%M:%S")
    
    if line_nr == skip:
        # find the time shift between incorrect and correct start time
        shift = true_time - time  # datetime.timedelta object
        print('Incorrect start time: ' + str(time))
        print('Time shift: ' + str(shift))
    
    corrected = time + shift
    line_new = sep.join([datetime.strftime(corrected, "%Y-%m-%d %H:%M:%S")] + cols[1:])
    lines_new.append(line_new)


with open(outfile, "w") as f:
    f.writelines(lines_new)
    
print("Time corrected data file '" + str(outfile) + "' has been produced.")

True start time: 2022-01-06 11:12:13
Incorrect start time: 2022-01-05 13:02:24
Time shift: 22:09:49
Time corrected data file '01061112_corrected_time.txt' has been produced.
